In [2]:
# Load model directly
from transformers import AutoProcessor, AutoModelForZeroShotImageClassification

processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")
model = AutoModelForZeroShotImageClassification.from_pretrained("openai/clip-vit-base-patch32")

2026-01-22 22:34:49.921383: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769121289.937335     735 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769121289.942259     735 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-22 22:34:49.958931: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencod

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [1]:

import torch
from PIL import Image
from transformers import CLIPModel, CLIPProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Load a multimodal encoder (image + text)
model_name = "openai/clip-vit-base-patch32"  # swap for OpenVision2-compatible CLIP if you want
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

model.eval()


2026-01-22 22:48:08.486420: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769122088.502191    1251 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769122088.507179    1251 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2026-01-22 22:48:08.523470: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/opt/conda/lib/python3.12/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencod

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e-05,

In [3]:

def embed_ad(image_path, headline, body_text, cta_text):
    """
    Returns a single ad embedding tensor of shape [hidden_dim]
    built from image + text components.
    """

    # 2. Load image
    image = Image.open(image_path).convert("RGB")

    # 3. Build a combined text description from components
    # You can change this template; keep it relatively stable.
    text_description = f"Ad with headline: {headline}. " \
                       f"Body: {body_text}. Call to action: {cta_text}."

    # 4. Preprocess for CLIP
    inputs = processor(
        text=[text_description],
        images=[image],
        return_tensors="pt",
        padding=True
    ).to(device)

    with torch.no_grad():
        outputs = model(**inputs)

        # 5. Get separate embeddings
        img_emb = outputs.image_embeds[0]   # shape [D]
        txt_emb = outputs.text_embeds[0]    # shape [D]

        # 6. Normalize (common practice with CLIP)
        img_emb = img_emb / img_emb.norm(p=2)
        txt_emb = txt_emb / txt_emb.norm(p=2)

        # 7. Fuse into a single ad embedding
        # Simple options: average, concatenation + small projection, gated sum, etc.
        # Here: weighted average as a very simple baseline.

        alpha = 0.6  # weight for image vs text; tune as needed
        ad_emb = alpha * img_emb + (1 - alpha) * txt_emb  # shape [D]

    return ad_emb.cpu()  # tensor of size [D], e.g., [512]


In [9]:
ad_embeddings = []
images = ["geeksforgeeks.png", 'geeksforgeeks2.png']

headlines = ["Summer Sale – 50% Off", "Shop now while supplies last", "Do not buy this coat!"]
bodies = ["Shop our new collection and save big on all items.", "If there was a time and a place, it is now", "I wish I could do that, you know?"]
ctas = ["Shop Now", "Shop Later", "Don't shop at all"]
for i in images:
    for h in headlines:
        for b in bodies:
            for c in ctas:
                ad_embeddings.append(embed_ad(i, h, b, c))


In [15]:
import random
ctr = [random.random() for x in range(len(ad_embeddings))]

In [16]:
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
X, y = make_friedman2(n_samples=500, noise=0, random_state=0)
kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,
        random_state=0).fit(ad_embeddings, ctr)


In [19]:
gpr.score(ad_embeddings, ctr)
m, s = gpr.predict(ad_embeddings, return_std=True)

In [20]:
s

array([0.2604907 , 0.25585604, 0.257458  , 0.25921304, 0.257563  ,
       0.25847684, 0.2571019 , 0.25714074, 0.25771796, 0.25807835,
       0.25706439, 0.25770012, 0.25759649, 0.25694568, 0.25769633,
       0.25768149, 0.25816749, 0.256717  , 0.2610328 , 0.25869559,
       0.25965106, 0.25754236, 0.25801726, 0.25941952, 0.25618228,
       0.25625995, 0.2588758 , 0.26046759, 0.25586651, 0.2574959 ,
       0.25923467, 0.25752704, 0.2585128 , 0.25714769, 0.25716236,
       0.25775288, 0.25809751, 0.25712932, 0.25775836, 0.25770503,
       0.25702991, 0.25776271, 0.2577718 , 0.25822418, 0.25677523,
       0.26083843, 0.25861617, 0.25957324, 0.25752992, 0.25796642,
       0.25938427, 0.25604624, 0.25615558, 0.25876423])

In [8]:
url = 'https://m.media-amazon.com/images/I/71RDJImfSKL._AC_SY625_.jpg'

import urllib.request
# from PIL import Image


urllib.request.urlretrieve(url, "geeksforgeeks2.png")

# # Opening the image and displaying it (to confirm its presence)
# img = Image.open(r"geeksforgeeks.png")
# img.show()

('geeksforgeeks2.png', <http.client.HTTPMessage at 0x7fddccd15010>)

torch.Size([512])
